In [1]:
import sys
import os
import shutil
import math
import numpy as np
import pandas as pd
import scipy.sparse as ss
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import time
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torchsummary import summary
import Metrics
# from STTN import *
from Param import *

In [2]:
# from folder workPEMSD7M

# MODEL CODE
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 28 10:28:06 2020

@author: wb
"""

import torch
import torch.nn as nn
# from GCN_models import GCN
# from One_hot_encoder import One_hot_encoder
import torch.nn.functional as F
import numpy as np
from scipy.sparse.linalg import eigs
import pandas as pd

DEVICE = 'cuda:0'

# def cheb_polynomial(L_tilde, K):
#     '''
#     compute a list of chebyshev polynomials from T_0 to T_{K-1}

#     Parameters
#     ----------
#     L_tilde: scaled Laplacian, np.ndarray, shape (N, N)

#     K: the maximum order of chebyshev polynomials

#     Returns
#     ----------
#     cheb_polynomials: list(np.ndarray), length: K, from T_0 to T_{K-1}

#     '''

#     N = L_tilde.shape[0]

#     cheb_polynomials = [np.identity(N), L_tilde.copy()]

#     for i in range(2, K):
#         cheb_polynomials.append(2 * L_tilde * cheb_polynomials[i - 1] - cheb_polynomials[i - 2])

#     return cheb_polynomials

# def scaled_Laplacian(W):
#     '''
#     compute \tilde{L}

#     Parameters
#     ----------
#     W: np.ndarray, shape is (N, N), N is the num of vertices

#     Returns
#     ----------
#     scaled_Laplacian: np.ndarray, shape (N, N)

#     '''

#     assert W.shape[0] == W.shape[1]

#     D = np.diag(np.sum(W, axis=1))

#     L = D - W

#     lambda_max = eigs(L, k=1, which='LR')[0].real

#     return (2 * L) / lambda_max - np.identity(W.shape[0])




# class GraphConvolution(nn.Module):
#     """
#     Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
#     """

#     def __init__(self, in_features, out_features, bias=True):
#         super(GraphConvolution, self).__init__()
#         self.in_features = in_features
#         self.out_features = out_features
#         self.weight = Parameter(torch.FloatTensor(in_features, out_features))
#         if bias:
#             self.bias = Parameter(torch.FloatTensor(out_features))
#         else:
#             self.register_parameter('bias', None)
#         self.reset_parameters()

#     def reset_parameters(self):
#         stdv = 1. / math.sqrt(self.weight.size(1))
#         self.weight.data.uniform_(-stdv, stdv)
#         if self.bias is not None:
#             self.bias.data.uniform_(-stdv, stdv)

#     def forward(self, x, adj):
#         support = torch.mm(x, self.weight)
#         output = torch.spmm(adj, support)
#         if self.bias is not None:
#             return output + self.bias
#         else:
#             return output

#     def __repr__(self):
#         return self.__class__.__name__ + ' (' \
#                + str(self.in_features) + ' -> ' \
#                + str(self.out_features) + ')'

    
# class cheb_conv(nn.Module):
#     '''
#     K-order chebyshev graph convolution
#     '''
#     def __init__(self, in_features, out_features, adj, K, bias=True):
# #     def __init__(self, nfeat, nhid, nclass, dropout, K):
# #     def __init__(self, K, cheb_polynomials, in_channels, out_channels):
#         '''
#         :param K: int
#         :param in_channles: int, num of channels in the input sequence
#         :param out_channels: int, num of channels in the output sequence
#         '''
#         super(cheb_conv, self).__init__()
#         self.DEVICE = DEVICE
#         self.K = K
#         adj = np.array(adj.cpu())
#         L_tilde = scaled_Laplacian(adj)
#         self.cheb_polynomials = [torch.from_numpy(i).type(torch.FloatTensor).to(self.DEVICE) for i in cheb_polynomial(L_tilde, K)]
        
        
#         self.in_channels = in_features
#         self.out_channels = out_features
        
#         self.Theta = nn.ParameterList([nn.Parameter(torch.randn(self.in_channels, self.out_channels).to(self.DEVICE)) for _ in range(K)])

#     def forward(self, x):
#         '''
#         Chebyshev graph convolution operation
#         :param x: (B, N, C] --> (batch_size, N, F_in, T)
#         :return: (batch_size, N, F_out, T)
#         '''
# #         x = x.permute(0, 1, 3, 2)
#         batch_size, num_of_vertices, in_channels = x.shape

           

#         graph_signal = x

#         output = torch.zeros(batch_size, num_of_vertices, self.out_channels).to(self.DEVICE)  # (b, N, F_out)

#         for k in range(self.K):

#             T_k = self.cheb_polynomials[k]  # (N,N)

#             theta_k = self.Theta[k]  # (in_channel, out_channel)

#             rhs = graph_signal.permute(0, 2, 1).matmul(T_k).permute(0, 2, 1) # （b, F_in, N) * (N, N) --> (b, F_in, N) --> (b, N, F_in)

#             output = output + rhs.matmul(theta_k) # (b, N, F_in) * (F_in, F_out) --> (b, N, F_out) 


        
#         result = F.relu(output)

#         return result

class One_hot_encoder(nn.Module):
    def __init__(self, embed_size, time_num = 288):
        super(One_hot_encoder, self).__init__()
        
        self.time_num = time_num
        self.I = nn.Parameter(torch.eye(time_num, time_num, requires_grad=True))
        self.onehot_Linear = nn.Linear(time_num, embed_size)     # 线性层改变one hot编码维度

    def forward(self, i, N = 25, T = 12):
    
        if i%self.time_num+T > self.time_num :
            o1 = self.I[i%self.time_num : , : ]
            o2 = self.I[0 : (i+T)%self.time_num, : ]
            onehot = torch.cat((o1, o2), 0)
        else:        
            onehot = self.I[i%self.time_num: i%self.time_num+T, : ]
        
        #onehot = onehot.repeat(N, 1, 1)   
        onehot = onehot.expand(N, T, self.time_num)
        onehot = self.onehot_Linear(onehot)
        return onehot
    

# class GCN(nn.Module):
#     def __init__(self, nfeat, nhid, nclass, adj, cheb_K, dropout):
#         super(GCN, self).__init__()

#         self.gc1 = cheb_conv(nfeat, nhid, adj, cheb_K)
#         self.gc2 = cheb_conv(nhid, nclass, adj, cheb_K)
#         self.dropout = dropout

#     def forward(self, x, adj):
#         x = F.relu(self.gc1(x))
#         x = F.dropout(x, self.dropout, training=self.training)
#         x = self.gc2(x)
#         return F.log_softmax(x, dim=1)


class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V):
        '''
        Q: [batch_size, n_heads, T(Spatial) or N(Temporal), N(Spatial) or T(Temporal), d_k]
        K: [batch_size, n_heads, T(Spatial) or N(Temporal), N(Spatial) or T(Temporal), d_k]
        V: [batch_size, n_heads, T(Spatial) or N(Temporal), N(Spatial) or T(Temporal), d_k]
        attn_mask: [batch_size, n_heads, seq_len, seq_len] 可能没有
        '''
        B, n_heads, len1, len2, d_k = Q.shape 
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) 
        # scores : [batch_size, n_heads, T(Spatial) or N(Temporal), N(Spatial) or T(Temporal), N(Spatial) or T(Temporal)]
        # scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is True.
        
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V) # [batch_size, n_heads, T(Spatial) or N(Temporal), N(Spatial) or T(Temporal), d_k]]
        return context



class SMultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SMultiHeadAttention, self).__init__()
        
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"
            
        # 用Linear来做投影矩阵    
        # 但这里如果是多头的话，是不是需要声明多个矩阵？？？

        self.W_V = nn.Linear(self.embed_size, self.head_dim * self.heads, bias=False)
        self.W_K = nn.Linear(self.embed_size, self.head_dim * self.heads, bias=False)
        self.W_Q = nn.Linear(self.embed_size, self.head_dim * self.heads, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)
    def forward(self, input_Q, input_K, input_V):
        '''
        input_Q: [batch_size, N, T, C]
        input_K: [batch_size, N, T, C]
        input_V: [batch_size, N, T, C]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
        B, N, T, C = input_Q.shape
#         print('input Q shape: ',input_Q.shape)
        # [B, N, T, C] --> [B, N, T, h * d_k] --> [B, N, T, h, d_k] --> [B, h, T, N, d_k]
        Q = self.W_Q(input_Q).view(B, N, T, self.heads, self.head_dim).transpose(1, 3)  # Q: [B, h, T, N, d_k]
        K = self.W_K(input_K).view(B, N, T, self.heads, self.head_dim).transpose(1, 3)  # K: [B, h, T, N, d_k]
        V = self.W_V(input_V).view(B, N, T, self.heads, self.head_dim).transpose(1, 3)  # V: [B, h, T, N, d_k]

        # attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]

        # context: [batch_size, n_heads, len_q, d_v], attn: [batch_size, n_heads, len_q, len_k]
        context = ScaledDotProductAttention()(Q, K, V) # [B, h, T, N, d_k]
        context = context.permute(0, 3, 2, 1, 4) #[B, N, T, h, d_k]
        context = context.reshape(B, N, T, self.heads * self.head_dim) # [B, N, T, C]
        # context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v) # context: [batch_size, len_q, n_heads * d_v]
        output = self.fc_out(context) # [batch_size, len_q, d_model]
        return output


class TMultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(TMultiHeadAttention, self).__init__()
        
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"
            
        # 用Linear来做投影矩阵    
        # 但这里如果是多头的话，是不是需要声明多个矩阵？？？

        self.W_V = nn.Linear(self.embed_size, self.head_dim * self.heads, bias=False)
        self.W_K = nn.Linear(self.embed_size, self.head_dim * self.heads, bias=False)
        self.W_Q = nn.Linear(self.embed_size, self.head_dim * self.heads, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)
    def forward(self, input_Q, input_K, input_V):
        '''
        input_Q: [batch_size, N, T, C]
        input_K: [batch_size, N, T, C]
        input_V: [batch_size, N, T, C]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
        B, N, T, C = input_Q.shape
        # [B, N, T, C] --> [B, N, T, h * d_k] --> [B, N, T, h, d_k] --> [B, h, N, T, d_k]
        Q = self.W_Q(input_Q).view(B, N, T, self.heads, self.head_dim).permute(0, 3, 1, 2, 4) # Q: [B, h, N, T, d_k]
        K = self.W_K(input_K).view(B, N, T, self.heads, self.head_dim).permute(0, 3, 1, 2, 4)  # K: [B, h, N, T, d_k]
        V = self.W_V(input_V).view(B, N, T, self.heads, self.head_dim).permute(0, 3, 1, 2, 4)  # V: [B, h, N, T, d_k]

        # attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]

        # context: [batch_size, n_heads, len_q, d_v], attn: [batch_size, n_heads, len_q, len_k]
        context = ScaledDotProductAttention()(Q, K, V) #[B, h, N, T, d_k]
        context = context.permute(0, 2, 3, 1, 4) #[B, N, T, h, d_k]
        context = context.reshape(B, N, T, self.heads * self.head_dim) # [B, N, T, C]
        # context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v) # context: [batch_size, len_q, n_heads * d_v]
        output = self.fc_out(context) # [batch_size, len_q, d_model]
        return output 



class STransformer(nn.Module):
    def __init__(self, embed_size, heads, adj, cheb_K, dropout, forward_expansion):
        super(STransformer, self).__init__()
        # Spatial Embedding
        self.adj = adj
        
#         print(adj)
#         print(adj.shape)
        
        self.D_S = adj.to(DEVICE) # 为什么是邻接矩阵
        self.embed_liner = nn.Linear(adj.shape[0], embed_size)
        
        self.attention = SMultiHeadAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )
        
        # 调用GCN
#         删掉GCN
#         self.gcn = GCN(embed_size, embed_size*2, embed_size, adj, cheb_K, dropout)  
        self.norm_adj = nn.InstanceNorm2d(1)    # 对邻接矩阵归一化

        self.dropout = nn.Dropout(dropout)
        self.fs = nn.Linear(embed_size, embed_size)
        self.fg = nn.Linear(embed_size, embed_size)

    def forward(self, value, key, query):
        # value, key, query: [N, T, C]  [B, N, T, C]        
        # Spatial Embedding 部分 
#         N, T, C = query.shape
#         D_S = self.embed_liner(self.D_S) # [N, C]
#         D_S = D_S.expand(T, N, C) #[T, N, C]相当于在第一维复制了T份
#         D_S = D_S.permute(1, 0, 2) #[N, T, C]
        B, N, T, C = query.shape
        D_S = self.embed_liner(self.D_S) # [N, C] 
        # 
        D_S = D_S.expand(B, T, N, C) #[B, T, N, C]相当于在第2维复制了T份, 第一维复制B份
        D_S = D_S.permute(0, 2, 1, 3) #[B, N, T, C]
        
        
#         # GCN 部分


#         X_G = torch.Tensor(B, N,  0, C).to(DEVICE)
#         self.adj = self.adj.unsqueeze(0).unsqueeze(0)
#         self.adj = self.norm_adj(self.adj)
#         self.adj = self.adj.squeeze(0).squeeze(0)
        
#         for t in range(query.shape[2]):
#             o = self.gcn(query[ : ,:,  t,  : ],  self.adj) # [B, N, C]
#             o = o.unsqueeze(2)              # shape [N, 1, C] [B, N, 1, C]
# #             print(o.shape)
#             X_G = torch.cat((X_G, o), dim=2)
#          # 最后X_G [B, N, T, C]   
        
#         print('After GCN:')
#         print(X_G)
        # Spatial Transformer 部分
        query = query + D_S
        
        
        
        
        attention = self.attention(query, query, query) #(B, N, T, C)
        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        U_S = self.dropout(self.norm2(forward + x))

        
        # 融合 STransformer and GCN  
#         g = torch.sigmoid(self.fs(U_S) +  self.fg(X_G))      # (7)
#         out = g*U_S + (1-g)*X_G                                # (8)
        
        out = U_S
        return out #(B, N, T, C)    


class TTransformer(nn.Module):
    def __init__(self, embed_size, heads, time_num, dropout, forward_expansion):
        super(TTransformer, self).__init__()
        
        # Temporal embedding One hot
        self.time_num = time_num
#         self.one_hot = One_hot_encoder(embed_size, time_num)          # temporal embedding选用one-hot方式 或者
        self.temporal_embedding = nn.Embedding(time_num, embed_size)  # temporal embedding选用nn.Embedding


        
        self.attention = TMultiHeadAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, t):
        B, N, T, C = query.shape
        
#         D_T = self.one_hot(t, N, T)                          # temporal embedding选用one-hot方式 或者
        D_T = self.temporal_embedding(torch.arange(0, T).to(DEVICE))    # temporal embedding选用nn.Embedding 
    # POSITON ENCODING
        D_T = D_T.expand(B, N, T, C)


        # temporal embedding加到query。 原论文采用concatenated
        query = query + D_T  
        
        attention = self.attention(query, query, query) # SELF-ATTENTION 的QKV都是一样，因为不是nlp

        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out




### STBlock

class STTransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, adj, time_num, cheb_K, dropout, forward_expansion):
        super(STTransformerBlock, self).__init__()
        self.STransformer = STransformer(embed_size, heads, adj, cheb_K, dropout, forward_expansion)
        self.TTransformer = TTransformer(embed_size, heads, time_num, dropout, forward_expansion)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, value, key, query, t):
    # value,  key, query: [N, T, C] [B, N, T, C]
        # Add skip connection,run through normalization and finally dropout
        x1 = self.norm1(self.STransformer(value, key, query) + query) #(B, N, T, C)
        x2 = self.dropout( self.norm2(self.TTransformer(x1, x1, x1, t) + x1) ) 
        return x2




### Encoder
class Encoder(nn.Module):
    # 堆叠多层 ST-Transformer Block
    def __init__(
        self,
        embed_size,
        num_layers,
        heads,
        adj,
        time_num,
        device,
        forward_expansion,
        cheb_K,
        dropout,
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.layers = nn.ModuleList(
            [
                STTransformerBlock(
                    embed_size,
                    heads,
                    adj,
                    time_num,
                    cheb_K,
                    dropout=dropout,
                    forward_expansion=forward_expansion
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, t):
    # x: [N, T, C]  [B, N, T, C]
        out = self.dropout(x)        
        # In the Encoder the query, key, value are all the same.
        for layer in self.layers:
            out = layer(out, out, out, t)
        return out     
    


### Transformer   
class Transformer(nn.Module):
    def __init__(
        self,
        adj,
        embed_size,
        num_layers,
        heads,
        time_num,
        forward_expansion, ##？
        cheb_K,
        dropout,
        
        device=DEVICE
    ):
        super(Transformer, self).__init__()
        self.encoder = Encoder(
            embed_size,
            num_layers,
            heads,
            adj,
            time_num,
            device,
            forward_expansion,
            cheb_K,
            dropout
        )
        self.device = device

    def forward(self, src, t): 
        ## scr: [N, T, C]   [B, N, T, C]
        enc_src = self.encoder(src, t) 
        return enc_src # [B, N, T, C]


### ST Transformer: Total Model

class STTransformer(nn.Module):
    def __init__(
        self, 
        adj,
        in_channels, 
        embed_size, 
        time_num,
        num_layers,
        T_dim,
        output_T_dim,  
        heads,    
        cheb_K,
        forward_expansion,
        dropout = 0
    ):        
        super(STTransformer, self).__init__()

        self.forward_expansion = forward_expansion
        # 第一次卷积扩充通道数
        
        # enbedded = head * d_k head是超参数
        
        self.conv1 = nn.Conv2d(in_channels, embed_size, 1)
        
        
        
        self.Transformer = Transformer(
            adj, # GCN用的
            embed_size, 
            num_layers, 
            heads, 
            time_num, # t-attention 中 position-encoding， 等于输入步长 time step-in == time num == 12
            forward_expansion,
            cheb_K, # GCN
            dropout = 0 # 剪枝
        )

        # 缩小时间维度。  例：T_dim=12到output_T_dim=3，输入12维降到输出3维
        self.conv2 = nn.Conv2d(T_dim, output_T_dim, 1)  
        # 缩小通道数，降到1维。
        self.conv3 = nn.Conv2d(embed_size, 1, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
#         platform: (CHANNEL, TIMESTEP_IN, N_NODE)
        
        
        # input x shape[ C, N, T] 
        # C:通道数量。  N:传感器数量。  T:时间数量
        
#         x = x.unsqueeze(0)
        
#         x = np.transpose(x,(0,2,1)).to(DEVICE)
        input_Transformer = self.conv1(x)        
#         input_Transformer = input_Transformer.squeeze(0)
#         input_Transformer = input_Transformer.permute(1, 2, 0)
        input_Transformer = input_Transformer.permute(0, 2, 3, 1)

        
        
        
        #input_Transformer shape[N, T, C]   [B, N, T, C]
        output_Transformer = self.Transformer(input_Transformer, self.forward_expansion)  # [B, N, T, C]
        output_Transformer = output_Transformer.permute(0, 2, 1, 3)
        #output_Transformer shape[B, T, N, C]
        
#         output_Transformer = output_Transformer.unsqueeze(0)     
        out = self.relu(self.conv2(output_Transformer))    # 等号左边 out shape: [1, output_T_dim, N, C]        
        out = out.permute(0, 3, 2, 1)           # 等号左边 out shape: [B, C, N, output_T_dim]
        out = self.conv3(out)                   # 等号左边 out shape: [B, 1, N, output_T_dim]   
#         out = out.squeeze(1)
        out = out.permute(0,1,3,2)
#         print('out: ',out.shape)
        return out #[B, N, output_dim]
        # return out shape: [N, output_dim]


def print_params(model_name, model):
    param_count=0
    for name, param in model.named_parameters():
        if param.requires_grad:
            param_count += param.numel()
    print(f'{model_name}, {param_count} trainable parameters in total.')
    return


    



    
    


In [3]:
v = pd.read_csv("../BEIJINGAIR/air_data/b0_V_10.csv", nrows = 35064, header= None)
v = np.array(v)

In [4]:
v.shape

(35064, 10)

In [5]:
cpu_num = 1
os.environ ['OMP_NUM_THREADS'] = str(cpu_num)
os.environ ['OPENBLAS_NUM_THREADS'] = str(cpu_num)
os.environ ['MKL_NUM_THREADS'] = str(cpu_num)
os.environ ['VECLIB_MAXIMUM_THREADS'] = str(cpu_num)
os.environ ['NUMEXPR_NUM_THREADS'] = str(cpu_num)
torch.set_num_threads(cpu_num)

In [12]:
DEVICE = 'cuda:0'
BATCHSIZE = 32
EPOCH = 200



def getXSYS(data, mode):
    TRAIN_NUM = int(data.shape[0] * TRAINRATIO)
    XS, YS = [], []
    if mode == 'TRAIN':    
        for i in range(TRAIN_NUM - TIMESTEP_OUT - TIMESTEP_IN + 1):
            x = data[i:i+TIMESTEP_IN, :]
            y = data[i+TIMESTEP_IN:i+TIMESTEP_IN+TIMESTEP_OUT, :]
            XS.append(x), YS.append(y)
    elif mode == 'TEST':
        for i in range(TRAIN_NUM - TIMESTEP_IN,  data.shape[0] - TIMESTEP_OUT - TIMESTEP_IN + 1):
            x = data[i:i+TIMESTEP_IN, :]
            y = data[i+TIMESTEP_IN:i+TIMESTEP_IN+TIMESTEP_OUT, :]
            XS.append(x), YS.append(y)
    XS, YS = np.array(XS), np.array(YS)
    XS, YS = XS[:, :, :, np.newaxis], YS[:, :, :, np.newaxis]
    XS = XS.transpose(0, 3, 2, 1)
    YS = YS.transpose(0, 3, 1, 2)
    print(XS.shape)
    print(YS.shape)
    
    return XS, YS



def getModel(name):
#     ks, kt, bs, T, n, p = 3, 3, [[CHANNEL, 16, 64], [64, 16, 64]], TIMESTEP_IN, N_NODE, 0
#     A = pd.read_csv(ADJPATH).values
#     W = weight_matrix(A)
#     L = scaled_laplacian(W)
#     Lk = cheb_poly(L, ks)
#     Lk = torch.Tensor(Lk.astype(np.float32)).to(device)
#     model = STGCN(ks, kt, bs, T, n, Lk, p).to(device)
#     return model
    
    ### Adjacency Matrix Import
#     adj_mx = pd.read_csv('../PEMSD7M/W_228.csv')#.iloc[:,1:]
#     adj_mx = np.array(adj_mx)
#     A = adj_mx
#     A = torch.Tensor(A)
    #ADJPATH = "../BEIJINGAIR/air_data/b_W_10.csv"
    A = pd.read_csv(ADJPATH,nrows = 10, header= None).values
    
#     print(ADJPATH)
#     print(A)
#     print(A.shape)
    
    
    A = torch.Tensor(A)


    ### Training Hyparameter
    in_channels = 1 # Channels of input
    embed_size = 64 # Dimension of hidden embedding features
    time_num = 24 
    num_layers = 2 # Number of ST Block
    T_dim = 12 # Input length, should be the same as prepareData.py
    output_T_dim = 12 # Output Expected length
    heads = 4 # Number of Heads in MultiHeadAttention
    cheb_K = 2 # Order for Chebyshev Polynomials (Eq 2)
    forward_expansion = 4 # Dimension of Feed Forward Network: embed_size --> embed_size * forward_expansion --> embed_size
    dropout = 0

    ### Construct Network
    model = STTransformer(
        A,
        in_channels, 
        embed_size, 
        time_num, 
        num_layers, 
        T_dim, 
        output_T_dim, 
        heads,
        cheb_K,
        forward_expansion,
        dropout).to(DEVICE)
    return model




    

def evaluateModel(model, criterion, data_iter):
    model.eval()
    l_sum, n = 0.0, 0
    with torch.no_grad():
        for x, y in data_iter:
            y_pred = model(x)
            l = criterion(y_pred, y)
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        return l_sum / n

def predictModel(model, data_iter):
    YS_pred = []
    model.eval()
    with torch.no_grad():
        for x, y in data_iter:
            YS_pred_batch = model(x)
            YS_pred_batch = YS_pred_batch.cpu().numpy()
            YS_pred.append(YS_pred_batch)
        YS_pred = np.vstack(YS_pred)
    return YS_pred

def trainModel(name, mode, XS, YS):
    print('Model Training Started ...', time.ctime())
    print('TIMESTEP_IN, TIMESTEP_OUT', TIMESTEP_IN, TIMESTEP_OUT)
    model = getModel(name)
#     summary(model, (CHANNEL,N_NODE,TIMESTEP_IN), device=device)
    XS_torch, YS_torch = torch.Tensor(XS).to(device), torch.Tensor(YS).to(device)
    trainval_data = torch.utils.data.TensorDataset(XS_torch, YS_torch)
    trainval_size = len(trainval_data)
    train_size = int(trainval_size * (1-TRAINVALSPLIT))
    print('XS_torch.shape:  ', XS_torch.shape)
    print('YS_torch.shape:  ', YS_torch.shape)
    train_data = torch.utils.data.Subset(trainval_data, list(range(0, train_size)))
    val_data = torch.utils.data.Subset(trainval_data, list(range(train_size, trainval_size)))
    train_iter = torch.utils.data.DataLoader(train_data, BATCHSIZE, shuffle=True)
    val_iter = torch.utils.data.DataLoader(val_data, BATCHSIZE, shuffle=True)
    
    min_val_loss = np.inf
    wait = 0

    print('LOSS is :',LOSS)
    if LOSS == "MaskMAE":
        criterion = Utils.masked_mae
    if LOSS == 'MSE':
        criterion = nn.MSELoss()
    if LOSS == 'MAE':
        criterion = nn.L1Loss()
    if OPTIMIZER == 'RMSprop':
        optimizer = torch.optim.RMSprop(model.parameters(), lr=LEARN)
    if OPTIMIZER == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=LEARN)
    
    for epoch in range(EPOCH):
        starttime = datetime.now()     
        loss_sum, n = 0.0, 0
        model.train()
        for x, y in train_iter:
            optimizer.zero_grad()
#             print(x.shape)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            loss_sum += loss.item() * y.shape[0]
            n += y.shape[0]
        train_loss = loss_sum / n
        val_loss = evaluateModel(model, criterion, val_iter)
        if val_loss < min_val_loss:
            wait = 0
            min_val_loss = val_loss
            torch.save(model.state_dict(), PATH + '/' + name + '.pt')
        else:
            wait += 1
            if wait == PATIENCE:
                print('Early stopping at epoch: %d' % epoch)
                break
        endtime = datetime.now()
        epoch_time = (endtime - starttime).seconds
        print("epoch", epoch, "time used:", epoch_time," seconds ", "train loss:", train_loss, "validation loss:", val_loss)
        with open(PATH + '/' + name + '_log.txt', 'a') as f:
            f.write("%s, %d, %s, %d, %s, %s, %.10f, %s, %.10f\n" % ("epoch", epoch, "time used", epoch_time, "seconds", "train loss", train_loss, "validation loss:", val_loss))
            
    torch_score = evaluateModel(model, criterion, train_iter)
    YS_pred = predictModel(model, torch.utils.data.DataLoader(trainval_data, BATCHSIZE, shuffle=False))
    print('YS.shape, YS_pred.shape,', YS.shape, YS_pred.shape)
    YS, YS_pred = scaler.inverse_transform(np.squeeze(YS)), scaler.inverse_transform(np.squeeze(YS_pred))
    print('YS.shape, YS_pred.shape,', YS.shape, YS_pred.shape)
    MSE, RMSE, MAE, MAPE = Metrics.evaluate(YS, YS_pred)
    with open(PATH + '/' + name + '_prediction_scores.txt', 'a') as f:
        f.write("%s, %s, Torch MSE, %.10e, %.10f\n" % (name, mode, torch_score, torch_score))
        f.write("%s, %s, MSE, RMSE, MAE, MAPE, %.10f, %.10f, %.10f, %.10f\n" % (name, mode, MSE, RMSE, MAE, MAPE))
    print('*' * 40)
    print("%s, %s, Torch MSE, %.10e, %.10f" % (name, mode, torch_score, torch_score))
    print("%s, %s, MSE, RMSE, MAE, MAPE, %.10f, %.10f, %.10f, %.10f" % (name, mode, MSE, RMSE, MAE, MAPE))
    print('Model Training Ended ...', time.ctime())
        
def testModel(name, mode, XS, YS):
    if LOSS == "MaskMAE":
        criterion = Utils.masked_mae
    if LOSS == 'MSE':
        criterion = nn.MSELoss()
    if LOSS == 'MAE':
        criterion = nn.L1Loss()
    print('Model Testing Started ...', time.ctime())
    print('TIMESTEP_IN, TIMESTEP_OUT', TIMESTEP_IN, TIMESTEP_OUT)
    XS_torch, YS_torch = torch.Tensor(XS).to(device), torch.Tensor(YS).to(device)
    test_data = torch.utils.data.TensorDataset(XS_torch, YS_torch)
    test_iter = torch.utils.data.DataLoader(test_data, BATCHSIZE, shuffle=False)
    model = getModel(name)
    model.load_state_dict(torch.load(PATH+ '/' + name + '.pt'))
    
    torch_score = evaluateModel(model, criterion, test_iter)
    YS_pred = predictModel(model, test_iter)
    print('YS.shape, YS_pred.shape,', YS.shape, YS_pred.shape)
    YS, YS_pred = scaler.inverse_transform(np.squeeze(YS)), scaler.inverse_transform(np.squeeze(YS_pred))
    print('YS.shape, YS_pred.shape,', YS.shape, YS_pred.shape)
    np.save(PATH + '/' + MODELNAME + '_prediction.npy', YS_pred)
    np.save(PATH + '/' + MODELNAME + '_groundtruth.npy', YS)
    MSE, RMSE, MAE, MAPE = Metrics.evaluate(YS, YS_pred)
    print('*' * 40)
    print("%s, %s, Torch MSE, %.10e, %.10f" % (name, mode, torch_score, torch_score))
    f = open(PATH + '/' + name + '_prediction_scores.txt', 'a')
    f.write("%s, %s, Torch MSE, %.10e, %.10f\n" % (name, mode, torch_score, torch_score))
    print("all pred steps, %s, %s, MSE, RMSE, MAE, MAPE, %.10f, %.10f, %.10f, %.10f" % (name, mode, MSE, RMSE, MAE, MAPE))
    f.write("all pred steps, %s, %s, MSE, RMSE, MAE, MAPE, %.10f, %.10f, %.10f, %.10f\n" % (name, mode, MSE, RMSE, MAE, MAPE))
    for i in range(TIMESTEP_OUT):
        MSE, RMSE, MAE, MAPE = Metrics.evaluate(YS[:, i, :], YS_pred[:, i, :])
        print("%d step, %s, %s, MSE, RMSE, MAE, MAPE, %.10f, %.10f, %.10f, %.10f" % (i+1, name, mode, MSE, RMSE, MAE, MAPE))
        f.write("%d step, %s, %s, MSE, RMSE, MAE, MAPE, %.10f, %.10f, %.10f, %.10f\n" % (i+1, name, mode, MSE, RMSE, MAE, MAPE))
    f.close()
    print('Model Testing Ended ...', time.ctime())

In [7]:
MODELNAME = 'STTN'
KEYWORD = 'pred_' + DATANAME + '_' + MODELNAME + '_' + datetime.now().strftime("%y%m%d%H%M")
PATH = '../save/' + KEYWORD
torch.manual_seed(100)
torch.cuda.manual_seed(100)
np.random.seed(100)
# torch.backends.cudnn.deterministic = True
########################################################### 
GPU = '0'
device = torch.device("cuda:{}".format(GPU)) if torch.cuda.is_available() else torch.device("cpu")
###########################################################
v = pd.read_csv("../BEIJINGAIR/air_data/b0_V_10.csv", nrows = 35064, header= None)
v = np.array(v)
v = v.T
v = torch.tensor(v, dtype=torch.float32)
v = v.transpose(1,0)
data = v
#data = pd.read_hdf(FLOWPATH).values
scaler = StandardScaler()
data = scaler.fit_transform(data)
print('data.shape', data.shape)

data.shape (35064, 10)


In [8]:
if not os.path.exists(PATH):
    os.makedirs(PATH)

In [9]:
print(KEYWORD, 'training started', time.ctime())
trainXS, trainYS = getXSYS(data, 'TRAIN')

pred_BEIJING-AIR_STTN_2201101911 training started Mon Jan 10 19:11:23 2022
(28028, 1, 10, 12)
(28028, 1, 12, 10)


In [10]:
print('TRAIN XS.shape YS,shape', trainXS.shape, trainYS.shape)

TRAIN XS.shape YS,shape (28028, 1, 10, 12) (28028, 1, 12, 10)


In [13]:
trainModel(MODELNAME, 'train', trainXS, trainYS)

Model Training Started ... Mon Jan 10 19:13:32 2022
TIMESTEP_IN, TIMESTEP_OUT 12 12
XS_torch.shape:   torch.Size([28028, 1, 10, 12])
YS_torch.shape:   torch.Size([28028, 1, 12, 10])
LOSS is : MAE
epoch 0 time used: 17  seconds  train loss: 0.4050197734361837 validation loss: 0.41374020203607814
epoch 1 time used: 18  seconds  train loss: 0.38149896095888275 validation loss: 0.4050885876428047
epoch 2 time used: 17  seconds  train loss: 0.3789183527372183 validation loss: 0.40481855158936486
epoch 3 time used: 17  seconds  train loss: 0.37719852010298854 validation loss: 0.40515188267241875


KeyboardInterrupt: 